In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import time
import random
import os
import logging
from datetime import datetime
import os
import sys
import requests
import pandas as pd
import re

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18844\1378756733.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def open_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)
def open_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        contents = f.read()
    return BeautifulSoup(contents, 'html.parser')

In [3]:
# 폴더 경로
folder_path = "../udemy/page/" # 실제 폴더 경로로 변경해주세요.

html_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.html')}
json_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.json')}


In [25]:
reviews_list = []
lecture_review_list = []
eeee = None
for pk in tqdm((html_files & json_files)):
    json_file = os.path.join(folder_path, f"{pk}.json")
    json_description = open_json(json_file)
    reviews = json_description["reviews"]
    new_site_review_count = 0
    new_site_review_rating = 0
    for r in reviews["results"]:
        if r["rating"] == None:
            continue
        temp={}
        temp["content"] = r["content"]
        temp["rating"] = r["rating"]
        temp["lecture_id"] = f"u{pk}"

        new_site_review_rating += temp["rating"]
        new_site_review_count += 1

        reviews_list.append(temp)
    lecture_review_list.append({
        "site_lecture_id" : f'u{pk}',
        "new_site_review_rating" : new_site_review_rating/(new_site_review_count*1.0) if new_site_review_count > 0 else 0,
        "new_site_review_count" : int(new_site_review_count)
    })

100%|██████████| 60130/60130 [00:57<00:00, 1045.36it/s]


In [30]:
pd.DataFrame(reviews_list).to_csv("../data-dump/udemy_review.csv",index=False)

In [27]:
pd.DataFrame(lecture_review_list).to_csv("../data-dump/review_rating_count.csv",index=False)

In [23]:
eeee
set(map(lambda x:x["rating"], eeee))

{0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, None}

In [10]:
for r in reviews["results"]:
    temp={}
    temp["content"] = r["content"]
    temp["rating"] = r["rating"]
    temp["lecture_id"] = f"u{pk}"
    reviews_list.append(temp)


{'content': '', 'rating': 4.5, 'lecture_id': 'u1672824'}


# 유데미 강의 테이블 리뷰 관련 수정

In [45]:
udemy_lectures = pd.read_csv("../udemy/lectures/udemy_lectures_dummy.csv")
udemy_lectures.head(1)


,site_lecture_id,name,image,category_name,price_original,price_sale,level,summary,description_summary,site_review_rating,site_review_count,site_student_count,review_sum,review_count,total_time,curriculum,instructor,site_type,site_link
0,u4452012,Powershell : Create a desktop application GUI,https://img-c.udemycdn.com/course/750x422/4452...,기타,20000,100000,INTERMEDIATE,Create a desktop application using powershell|...,Develop a standalone application Tool using Po...,4.112416,26,2206,0,0,241,"{'curriculum': [{'time': 4, 'item_count': 1, '...",Vishnu Gupthan Nair,UDEMY,https://udemy.com/course/powershell-create-a-d...


In [46]:
temp = udemy_lectures["price_sale"].values
udemy_lectures["price_sale"] = udemy_lectures["price_original"]
udemy_lectures["price_original"] = temp

In [43]:
review_rating_count = pd.read_csv("../data-dump/review_rating_count.csv")
review_rating_count

,site_lecture_id,new_site_review_rating,new_site_review_count
0,u1672824,4.307547,265
1,u5491806,4.500000,96
2,u4705372,4.333333,3
3,u1895050,4.394231,52
4,u3087732,4.318750,80
...,...,...,...
60125,u2089610,4.315217,92
60126,u3314248,4.500000,2
60127,u5684170,0.000000,0
60128,u4151466,4.401243,724


In [47]:
udemy_lectures = pd.merge(left=udemy_lectures, right=review_rating_count, how="left", left_on="site_lecture_id", right_on="site_lecture_id")
udemy_lectures = udemy_lectures.drop(columns=["site_review_rating","site_review_count"])
udemy_lectures

,site_lecture_id,name,image,category_name,price_original,price_sale,level,summary,description_summary,site_student_count,review_sum,review_count,total_time,curriculum,instructor,site_type,site_link,new_site_review_rating,new_site_review_count
0,u4452012,Powershell : Create a desktop application GUI,https://img-c.udemycdn.com/course/750x422/4452...,기타,100000,20000,INTERMEDIATE,Create a desktop application using powershell|...,Develop a standalone application Tool using Po...,2206,0,0,241,"{'curriculum': [{'time': 4, 'item_count': 1, '...",Vishnu Gupthan Nair,UDEMY,https://udemy.com/course/powershell-create-a-d...,4.346154,26
1,u3429610,Complete Core Java for Testers+Interview Programs,https://img-c.udemycdn.com/course/750x422/3429...,프로그래밍 언어,25000,15000,ALL,You will get comprehensive knowledge of Java P...,Best course on Java Programming for complete b...,70,0,0,488,"{'curriculum': [{'time': 5, 'item_count': 1, '...",Tech Planet,UDEMY,https://udemy.com/course/complete-core-java-fo...,4.309524,21
2,u5168890,CompTIA A+ Core 2 (220-1102) Exam Questions an...,https://img-c.udemycdn.com/course/750x422/5168...,자격증,25000,15000,ALL,Exam Domain 1.0 Operating Systems - 31%||Exam ...,Pass A+ Certification Core 2 (220-1102) Exam G...,2,0,0,0,"{'curriculum': [{'time': 0, 'item_count': 0, '...",Birali Faustin Nzubahimana,UDEMY,https://udemy.com/course/comptia-a-core-2-220-...,0.000000,0
3,u2119890,الدليل الشامل لتعلم بايثون 2022,https://img-c.udemycdn.com/course/750x422/2119...,프로그래밍 언어,25000,15000,ALL,هنعرف يعني ايه برمجه||هنتعرف علي تاريخ لغه برم...,الدليل الشامل لتعلم البرمجه ببايثون من الصفر ب...,25258,0,0,537,"{'curriculum': [{'time': 2, 'item_count': 4, '...",Mahmoud Ahmed,UDEMY,https://udemy.com/course/learn-python-arabic/,4.402680,1567
4,u1740734,HAProxy for Beginners,https://img-c.udemycdn.com/course/750x422/1740...,기타,25000,25000,ALL,Students will be able to build a high availabl...,Load Balancing and High Availability,2964,0,0,102,"{'curriculum': [{'time': 102, 'item_count': 16...",Hadi Alnabriss,UDEMY,https://udemy.com/course/haproxy-a/,4.173212,713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59530,u5129420,취업 준비생을 위한 Git (소스코드 버전관리),https://img-c.udemycdn.com/course/750x422/5129...,웹 개발,25000,15000,BEGINNER,소스 코드 형상 관리 기본 이해||macOS / Windows 터미널 명령어 이해|...,"[2023 취업대비] 디자인, 퍼블리싱 직군 취업 준비생을 위해 실습을 구성하였습니다.",173,0,0,41,"{'curriculum': [{'time': 7, 'item_count': 2, '...",황선수닷컴 Hwang Sunsoo,UDEMY,https://udemy.com/course/command-line-git-basic/,4.593750,80
59531,u4083098,Create a Smart Contract on WAX blockchain incl...,https://img-c.udemycdn.com/course/750x422/4083...,프로그래밍 언어,70000,20000,INTERMEDIATE,How to setup a environment in Docker to develo...,Learn how to interact with blockchain Oracles ...,618,0,0,306,"{'curriculum': [{'time': 3, 'item_count': 1, '...",Icaro Lavrador,UDEMY,https://udemy.com/course/wax-blockchain/,4.038095,105
59532,u1181318,Learn 2D Character Animation for Games with Bl...,https://img-c.udemycdn.com/course/750x422/1181...,게임 개발,70000,20000,INTERMEDIATE,Learn how to model and texture map 2D Characte...,"Explore how to build, animate, and setup 2D ga...",521,0,0,296,"{'curriculum': [{'time': 60, 'item_count': 12,...",3dmotive LLC,UDEMY,https://udemy.com/course/3dmotive-learn-2d-cha...,4.229730,37
59533,u3772896,"Weblogic Performance Tuning, Patching & Upgrad...",https://img-c.udemycdn.com/course/750x422/3772...,자격증,65000,15000,ALL,Fundamentals of performance tuning||Basic Term...,The Complete Industrial Approach For Certifica...,238,0,0,201,"{'curriculum': [{'time': 31, 'item_count': 5, ...",Sandip Jha,UDEMY,https://udemy.com/course/weblogic-performance-...,3.982759,58


In [48]:
udemy_lectures.rename(columns = {'new_site_review_rating' : 'site_review_rating'}, inplace = True)
udemy_lectures.rename(columns = {'new_site_review_count' : 'site_review_count'}, inplace = True)
udemy_lectures.columns

Index(['site_lecture_id', 'name', 'image', 'category_name', 'price_original',
       'price_sale', 'level', 'summary', 'description_summary',
       'site_student_count', 'review_sum', 'review_count', 'total_time',
       'curriculum', 'instructor', 'site_type', 'site_link',
       'site_review_rating', 'site_review_count'],
      dtype='object')

In [49]:
udemy_lectures.to_csv("../udemy/lectures/udemy_lectures_dummy.csv",index=False)

In [ ]:
for i in udemy_lectures["site_lecture_id"].values:
    if i not in 